In [4]:
# mei ref: https://music-encoding.org/guidelines/v4/data-types/data.accidental.written.html

In [10]:
import numpy as np
import pandas as pd
from typing import List, Tuple
from IPython.display import display, HTML
import verovio

# Adjustable Variables
A4_FREQUENCY = 440
FILE_PATH = 'conga3.tsv'
STAFF = 'GF'
VEROVIO_OPTIONS = {
    "inputFormat": "mei",
    "scale": 50,
    "adjustPageHeight": True,
    "noHeader": True,
    "border": 0,
    "pageHeight": 1300,
    "pageWidth": 1800,
    "staffLineWidth": 1.5,
    "systemDividerLineWidth": 1.5
}

DISPLAY_MAX_ROWS = None

# Constants
PITCH_NAMES = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]

def convert_frequencies(frequencies: List[float]) -> Tuple[List[str], List[int], List[str]]:
    log_freq_ratios = [np.log2(freq / A4_FREQUENCY) for freq in frequencies]
    midi_values = [int(round(12 * log_freq_ratio + 69)) for log_freq_ratio in log_freq_ratios]
    midi_cent_values = [int(round(69 * 100 + 1200 * log_freq_ratio)) for log_freq_ratio in log_freq_ratios]
    pitch_names = [f"{PITCH_NAMES[midi_value % 12]}{(midi_value - 12) // 12}" for midi_value in midi_values]
    midi_cent_deviations = [((midi_cent_value - midi_value * 100) + 50) % 100 - 50 for midi_value, midi_cent_value in zip(midi_values, midi_cent_values)]
    midi_cent_deviations = [f'+{x}' if x >= 0 else f'{x}' for x in midi_cent_deviations]

    return pitch_names, midi_values, midi_cent_deviations

def pitch_name_and_cent_deviation_to_step_octave_alter(pitch_names: List[str], midi_cent_deviations: List[str]) -> List[Tuple[str, int, str]]:
    notes_and_alters = []
    for pitch_name, cent_deviation in zip(pitch_names, midi_cent_deviations):
        step = pitch_name[0]
        octave = int(pitch_name[-1])
        deviation = int(cent_deviation)
        alter = 'n'
        if '#' in pitch_name:
            alter = 's'
            if deviation >= 25:
                alter = '3qs'
            elif deviation > 0:
                alter = '1qs'
        elif 'b' in pitch_name:
            alter = 'f'
            if deviation <= -25:
                alter = '3qf'
            elif deviation < 0:
                alter = '1qf'
        else:
            if deviation >= 25:
                alter = '1qs'
            elif deviation > 0:
                alter = '1qs'
            elif deviation <= -25:
                alter = '1qf'
            elif deviation < 0:
                alter = '1qf'
        notes_and_alters.append((step, octave, alter))
    return notes_and_alters

def create_mei_string(notes_and_alters: List[Tuple[str, int, str]], midi_values: List[int], staff: str) -> str:
    staff_defs = ''
    if 'G' in staff:
        staff_defs += '<staffDef n="1" lines="5" clef.shape="G" clef.line="2"/>\n'
    if 'F' in staff:
        staff_defs += '<staffDef n="2" lines="5" clef.shape="F" clef.line="4"/>\n'
    
    measures = []
    for ((step, octave, alter), midi_value) in zip(notes_and_alters, midi_values):
        measure = '<measure right="invis">\n'
        if 'G' in staff:
            measure += '<staff n="1">\n<layer n="1">\n'
            if octave >= 4 or (octave == 3 and step == 'B'):
                measure += f'''
                                        <note dur="4" oct="{octave}" pname="{step.lower()}" pnum="{midi_value}" stem.dir="down" accid="{alter}" stem.visible="false"/>
                '''
            measure += '</layer>\n</staff>\n'
        if 'F' in staff:
            measure += '<staff n="2">\n<layer n="1">\n'
            if octave < 4 or (octave == 4 and step == 'C' and alter in ['f', '1qf', '3qf']):
                measure += f'''
                                        <note dur="4" oct="{octave}" pname="{step.lower()}" pnum="{midi_value}" stem.dir="down" accid="{alter}" stem.visible="false"/>
                '''
            measure += '</layer>\n</staff>\n'
        measure += '</measure>\n'
        measures.append(measure)
    
    mei = f'''
    <?xml version="1.0" encoding="UTF-8"?>
    <mei xmlns="http://www.music-encoding.org/ns/mei" meiversion="4.0.1">
        <music>
            <body>
                <mdiv>
                    <score>
                        <scoreDef keysig="0" mode="major">
                            <staffGrp n="1">
                                {staff_defs}
                            </staffGrp>
                        </scoreDef>
                        <section>
                            {''.join(measures)}
                        </section>
                    </score>
                </mdiv>
            </body>
        </music>
    </mei>
    '''
    return mei


# Main Script
df = pd.read_csv(FILE_PATH, sep='\t')
frequencies = df['Frequency (Hz)'].tolist()

pitch_names, midi_values, midi_cent_deviations = convert_frequencies(frequencies)
notes_and_alters = pitch_name_and_cent_deviation_to_step_octave_alter(pitch_names, midi_cent_deviations)
mei = create_mei_string(notes_and_alters, midi_values, STAFF)

# create a DataFrame
df_para = pd.DataFrame({
    'Frequency (Hz)': frequencies,
    'MIDI Value': midi_values,
    'MIDI Cent Deviation': midi_cent_deviations,
    'Pitch Name': pitch_names,
    'Note': [x[0] for x in notes_and_alters],
    'Octave': [x[1] for x in notes_and_alters],
    'Alter': [x[2] for x in notes_and_alters],
})

pd.options.display.max_rows = DISPLAY_MAX_ROWS
display(df_para)

# Create a VerovioToolkit object
vrvToolkit = verovio.toolkit()

# Render the music notation and get the HTML code
htmlCode = vrvToolkit.renderData(mei, options=VEROVIO_OPTIONS)

# Display the HTML code in the notebook
display(HTML(htmlCode))


,Frequency (Hz),MIDI Value,MIDI Cent Deviation,Pitch Name,Note,Octave,Alter
0,31.966817,24,-39,C1,C,1,1qf
1,75.181218,38,+41,D2,D,2,1qs
2,126.683312,47,+44,B2,B,2,1qs
3,163.977932,52,-9,E3,E,3,1qf
4,181.737274,54,-31,F#3,F,3,s
5,189.432990,54,+41,F#3,F,3,3qs
6,239.159149,58,+45,A#3,A,3,3qs
7,282.965528,61,+36,C#4,C,4,3qs
8,300.724871,62,+41,D4,D,4,1qs
9,321.444104,64,-44,E4,E,4,1qf
